# Auricular shape analysis - age prediction

In [ ]:
%load_ext autoreload
%autoreload 2

import logging
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import trimesh
from PIL import Image
from scipy.ndimage import distance_transform_edt, binary_fill_holes

In [ ]:
sys.path.append("/home/vajicek/src/aurikular/shapeexplorer")

from projects.auricular.projection import computeHeightmap, regularSampling, getDistanceToEdge
from projects.auricular.curvaturedescriptors import CurvatureDescriptorsParams
from projects.auricular.curvaturedescriptors import CurvatureDescriptors
from projects.auricular.curvaturedescriptors import HistogramDescriptors
from projects.auricular.curvaturedescriptors import ModelAnalysis
from projects.auricular.common import getSample

In [ ]:
input_data = os.path.expanduser("~/data/aurikularni_plocha_ply5/")
sample = list(getSample(input_data))
params = CurvatureDescriptorsParams(
    input_data=input_data,
    sampling_method='regular',
    dist=1.0,
    sampling_rate=0.5,
    sample_count=5000,
    output='../../../output_812')
cd = CurvatureDescriptors(params)

In [ ]:
data = cd.getData()
hist_descriptors = {
    0.5: HistogramDescriptors(data, 0.5),
    1.0: HistogramDescriptors(data, 1.0),
    2.0: HistogramDescriptors(data, 2.0)
}

## Prediction

### age x mean dne

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.svm import LinearSVR

from projects.auricular.analyze import evaluateAllModels

In [ ]:
df = pd.DataFrame(data=[[
                    float(data1['age']),
                    np.log(float(data1['age'])),
                    float(data1['dist'][1.0]['sampled_dne'])] for data1 in data],
                  columns = ['age', 'logAge', 'dne'])

r = evaluateAllModels(df, indeps=[['dne']], dep=['logAge'], model=LinearRegression())
pd.DataFrame(r)

### age x vector dne

In [ ]:
pd.DataFrame(hist_descriptors[0.5].getSampleHistogramData(3)[0]).plot.bar()
pd.DataFrame(hist_descriptors[0.5].getSampleHistogramData(3)[249]).plot.bar()
pd.DataFrame(hist_descriptors[0.5].getSampleHistogramData(3)[498]).plot.bar();

In [ ]:
ma=ModelAnalysis(data, 'dist_curv')

In [ ]:
ma.twoParamPlot(1.0)

In [ ]:
ma.modelForBins(3, None, 1.0, model=SVR(), normalize_dist=False)

In [ ]:
ma.modelForBins(3, [[0, 2]], 1.0)

In [ ]:
ma.modelForBins(3, [[0, 1]], 1.0)

In [ ]:
ma.plotRmsePerBins(list(ma.binsRmse(2.0)))

In [ ]:
ma.plotRmsePerBins(list(ma.binsRmse(1.0, model=SVR())))

In [ ]:
ma.plotRmsePerBins(list(ma.binsRmse(model=LinearSVR())))

In [ ]:
ma.compareMethods(2.0)

In [ ]:
ma.compareMethods(1.0)

In [ ]:
ma.compareMethods(0.5)

## Neural network

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

In [ ]:
dist=1.0
bins=10

Y = pd.DataFrame([np.log(float(data1['age'])) for data1 in data]).values
X = pd.DataFrame(ma.hist_descriptors[dist].getSampleHistogram2dData(bins, True, True)).values
#X = pd.DataFrame(hist_descriptors[dist].getSampleHistogramData(bins)).values

In [ ]:
inputs = X.shape[1]
def baselineModel():
    model = keras.Sequential()
    model.add(Dense(inputs,
                    input_dim=inputs,
                    kernel_initializer='he_normal', 
                    activation='relu'))
    model.add(Dense(inputs, kernel_initializer='he_normal', activation='relu'))
    model.add(Dense(inputs, kernel_initializer='he_normal', activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.00005,
                                                  beta_1=0.95,
                                                  beta_2=0.9995),
                  loss='mean_squared_error')
    return model

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=None)

# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

model = baselineModel()
model.fit(X_train, y_train,
          use_multiprocessing=True,
          epochs=250,
          batch_size=10,
          validation_data=(X_test, y_test))

In [ ]:
predictions = model.predict(X_test)
np.sqrt(mean_squared_error(np.exp(predictions), np.exp(y_test)))

#np.exp(model.evaluate(X_test, y_test))

In [ ]:
model.evaluate(X_test, y_test)
#y_test

In [ ]:
def evaluatePipeline(X, Y):   
    estimators = []
    estimators.append(('standardize', StandardScaler()))
    estimators.append(('mlp', KerasRegressor(build_fn=baselineModel,
                                      epochs=1000,
                                      batch_size=20,
                                      verbose=0)))
    pipeline = Pipeline(estimators)

    kf = KFold(n_splits=10, shuffle=True)
    fold_rmse = []
    predicted = np.array([])
    predicted_indices = np.array([])
    for train_index, test_index in kf.split(X):
        X_train = X[train_index]
        X_test = X[test_index]
        y_train = Y[train_index]
        y_test = Y[test_index]
        pipeline.fit(X_train, y_train)
        predictions = pipeline.predict(X_test)
        rmse = np.sqrt(mean_squared_error(np.exp(predictions), np.exp(y_test)))
        fold_rmse += [rmse]
        print(rmse)
        predicted = np.append(predicted, predictions)
        predicted_indices = np.append(predicted_indices, test_index)

    return np.mean(fold_rmse), predicted, predicted_indices

In [ ]:
X = pd.DataFrame(hist_descriptors[dist].getSampleHistogramData(bins)).values
result_curv = evaluatePipeline(X, Y)

In [ ]:
X = pd.DataFrame(ma.hist_descriptors[dist].getSampleHistogram2dData(bins, True, True)).values
result_curv_dist = evaluatePipeline(X, Y)

In [ ]:
rmse, predicted, predicted_indices = result_curv_dist
rmse

In [ ]:
fig, ax = plt.subplots(figsize=(15,15))
plt.scatter(np.exp(Y[predicted_indices.astype(int)]), np.exp(predicted))
plt.xlabel("actual")
plt.ylabel("predicted")
plt.plot([20, 100], [20, 100], color="black", linewidth=1);